In [1]:
import numpy as np
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math 


In [2]:
def model(A, THETA):
    """
    Multivariate Linear Regression Model, Yh = A*THETA
    The matrix A is sometimes called the design matrix.
    """
    return A.dot(THETA)

In [3]:
# Design Matrix
def designMatrix(Tau,X):
    q,_ = X.shape
    for p in range(q):
        M = powerVector(Tau,X[p,:])
        if p == 0:
            A = M
            continue
        A = np.vstack((A,M))
    return A

# Power Vector M
# def powerVector(Tau, V):
#     if V.size == 0 or Tau == 0:
#         return np.array([1.0])
#     Z = V[:-1]
#     W = V[-1]
#     terms = []
#     for k in range(Tau + 1):
#         sub_terms = powerVector(Tau - k, Z)
#         for term in sub_terms:
#             terms.append(term * (W ** k))
#     return np.array(terms)

def powerVector(Tau, V):
    if V.size == 0 or Tau == 0:
        return np.array([[1.0]])  # Asegurar 2D
    Z = V[:-1]
    W = V[-1]
    terms = []
    for k in range(Tau + 1):
        sub_terms = powerVector(Tau - k, Z)
        for term in sub_terms.flatten():  # Manejar sub_terms 2D
            terms.append(term * (W ** k))
    return np.array([terms])  # Devolver como 2D (1, n_terms)


In [4]:
#Polynomial parameter number
# def polyParamsNumber(n,tau):
#     s = 0
#     for l in range(tau+1):
#         #val = np.math.factorial(l+n-1)/np.math.factorial(n-1)
#         val = math.factorial(l + n - 1) / (math.factorial(n - 1) * math.factorial(l))
#         val = val/np.math.factorial(l)
#         s = s + val
#     return int(s)


# def polyParamsNumber(n,tau):
#     s = 0
#     for l in range(tau+1):
#         val = math.factorial(l + n - 1) / (math.factorial(n - 1) * math.factorial(l))
#         val = val / math.factorial(l) 
#         s += val
#     return int(s)

def polyParamsNumber(n, tau):
    return int(math.comb(n + tau, tau))




In [5]:
def loss(Y_true, Y_pred, THETA, lambda_param):
    """ Mean Squared Error """
    E = Y_true - Y_pred
    SSE = np.square(np.linalg.norm(E, 'fro'))
    Reg = (lambda_param/(2*E.shape[0]))*np.square(np.linalg.norm(THETA[1:,:], 'fro'))
    MSE = SSE/(2*E.shape[0]) + Reg
    return MSE


In [6]:
def gradient(A,E,THETA,lambda_param):
    """ MSE Gradient """
    SSEGrad = -1.0*(A.T).dot(E)
    MSEGrad = SSEGrad/E.shape[0]
    MSEGrad[1:,:] = MSEGrad[1:,:] + (lambda_param/E.shape[0])*THETA[1:,:]
    return MSEGrad




In [7]:
#GDX
def gdx_optimization(
    X,
    Y,
    tau,
    lambda_param=0.0,
    maxEpochs=100,
    show=10,
    batch_size=16,
    learning_rate=0.01,
    momentum=0.9,
    lr_dec=0.5,
    lr_inc=1.05,
    max_perf_inc=1.04,
    stopping_threshold=1e-6,
):
    """Optimizador GDX con manejo de tamaño de lote variable"""
    n = X.shape[1]
    m = Y.shape[1]
    rho = polyParamsNumber(n, tau)
    #rho me esta dando porbelmas 
    THETA = np.random.randn(rho, m) *0.01 #reduce la escala de la inicalizaciones de los pesos
    delta_THETA = np.zeros((rho, m))
    lr = learning_rate
    q = X.shape[0]
    previous_loss = np.inf

    for epoch in range(maxEpochs + 1):
        THETA_prev = THETA.copy()
        current_loss = np.inf

        if batch_size < q:
            indices = np.random.permutation(q)
            X = X[indices, :]
            Y = Y[indices, :]

        n_batches = q // batch_size
        residual = q % batch_size
        total_batches = n_batches + 1 if residual != 0 else n_batches

        for batch_idx in range(total_batches):
            start = batch_idx * batch_size
            end = start + batch_size
            if batch_idx == total_batches - 1 and residual != 0:
                end = start + residual

            X_batch = X[start:end, :]
            Y_batch = Y[start:end, :]
            # A_batch = designMatrix(tau, X_batch)
            # Y_pred = model(A_batch, THETA)
            # E_batch = Y_batch - Y_pred

            A_batch = designMatrix(tau, X_batch)  # (batch_size, rho)
            Y_pred = model(A_batch, THETA)  # (batch_size, m)
            E_batch = Y_batch - Y_pred  # (batch_size, m)
            Grad = gradient(A_batch, E_batch, THETA, lambda_param)
            #evitar desbordamienteos 
            Grad = np.clip(Grad, -1e3, 1e3)
            delta_THETA = momentum * delta_THETA - (1 - momentum) * lr * Grad
            THETA += delta_THETA

        A = designMatrix(tau, X)
        Yh = model(A, THETA)
        current_loss = loss(Y, Yh, THETA, lambda_param)

        if current_loss > previous_loss * max_perf_inc:
            THETA = THETA_prev
            lr *= lr_dec
        elif current_loss < previous_loss:
            lr *= lr_inc

        if epoch % show == 0:
            print(f"Epoch {epoch}: Loss={current_loss:.3e}, lr={lr:.2e}")
        if abs(previous_loss - current_loss) < stopping_threshold:
            break
        previous_loss = current_loss

    return THETA

In [8]:
# Load dataset
mat = sp.loadmat('engine_dataset.mat')
#print(mat.keys())
inputs  = mat['engineInputs'].T
targets = mat['engineTargets'].T


In [9]:
# Train and Test Split Data
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, random_state = 1, test_size = 0.4)




In [10]:
# Train and Test Data
# Train Data
xTrain = inputs_train
tTrain = targets_train
# Test Data
xTest = inputs_test
tTest = targets_test



In [11]:

#mini batch
# Find the optimal parameters m and b with RMSprop
#polinomio grado 1
# tau = 1
# lambda_param = 0.1

# THETA = gdx_optimization(
#     xTrain,
#     tTrain,
#     tau=tau,
#     lambda_param=lambda_param,
#     maxEpochs=10000,
#     show=500,
#     batch_size=256,
#     learning_rate=1e-8,
#     momentum=0.9,          
#     lr_dec=0.5,           
#     lr_inc=1.05,
#     max_perf_inc=1.04,
#     stopping_threshold=1e-6,
# )


#vamos a ver con el grado 2
tau = 2
lambda_param = 0.01

THETA = gdx_optimization(
    xTrain,
    tTrain,
    tau=tau,
    lambda_param=lambda_param,
    maxEpochs=5000,
    show=100,
    batch_size=64,
    learning_rate=1e-8,
    momentum=0.9,          
    lr_dec=0.5,           
    lr_inc=1.05,
    max_perf_inc=1.04,
    stopping_threshold=1e-6,
)

Epoch 0: Loss=3.905e+07, lr=1.05e-08
Epoch 100: Loss=1.525e+05, lr=3.80e-08
Epoch 200: Loss=1.458e+05, lr=6.96e-08
Epoch 300: Loss=1.400e+05, lr=2.75e-08
Epoch 400: Loss=1.356e+05, lr=4.56e-08
Epoch 500: Loss=1.312e+05, lr=5.65e-08
Epoch 600: Loss=1.270e+05, lr=4.46e-08
Epoch 700: Loss=1.234e+05, lr=3.53e-08
Epoch 800: Loss=1.187e+05, lr=4.81e-08
Epoch 900: Loss=1.146e+05, lr=3.29e-08
Epoch 1000: Loss=1.113e+05, lr=2.60e-08
Epoch 1100: Loss=1.115e+05, lr=6.43e-08
Epoch 1200: Loss=1.047e+05, lr=2.54e-08
Epoch 1300: Loss=1.023e+05, lr=3.47e-08
Epoch 1400: Loss=1.001e+05, lr=4.97e-08
Epoch 1500: Loss=9.805e+04, lr=3.93e-08
Epoch 1600: Loss=9.396e+04, lr=5.91e-08
Epoch 1700: Loss=9.668e+04, lr=3.66e-08
Epoch 1800: Loss=9.137e+04, lr=3.04e-08
Epoch 1900: Loss=9.044e+04, lr=3.76e-08
Epoch 2000: Loss=8.705e+04, lr=3.28e-08
Epoch 2100: Loss=8.491e+04, lr=2.47e-08
Epoch 2200: Loss=8.270e+04, lr=3.71e-08
Epoch 2300: Loss=8.156e+04, lr=5.59e-08
Epoch 2400: Loss=7.956e+04, lr=4.00e-08
Epoch 2500: 

In [12]:

#ONLINE (1 a 1)
"""
tau = 2
lambda_param = 0.001
THETA = gdx_optimization(
    xTrain,
    tTrain,
    tau=tau,
    lambda_param=lambda_param,
    maxEpochs=100000,
    show=1000,
    batch_size=1,
    learning_rate=1e-6,
    momentum=0.9,          
    lr_dec=0.5,           
    lr_inc=1.05,
    max_perf_inc=1.04,
    stopping_threshold=1e-6,
)"""


'\ntau = 2\nlambda_param = 0.001\nTHETA = gdx_optimization(\n    xTrain,\n    tTrain,\n    tau=tau,\n    lambda_param=lambda_param,\n    maxEpochs=100000,\n    show=1000,\n    batch_size=1,\n    learning_rate=1e-6,\n    momentum=0.9,          \n    lr_dec=0.5,           \n    lr_inc=1.05,\n    max_perf_inc=1.04,\n    stopping_threshold=1e-6,\n)'

In [13]:

# #lote
# tau = 1
# lambda_param = 0.001
# THETA = gdx_optimization(
#     xTrain,
#     tTrain,
#     tau=tau,
#     lambda_param=lambda_param,
#     maxEpochs=10000,
#     show=500,
#     batch_size=xTrain.shape[0],
#     learning_rate=1e-4,
#     momentum=0.9,          
#     lr_dec=0.5,           
#     lr_inc=1.05,
#     max_perf_inc=1.04,
#     stopping_threshold=1e-6,
# )


In [14]:
# Make predictions
# Train data
A_train = designMatrix(tau,xTrain)
outputTrain = model(A_train,THETA)
# Test data
A_test = designMatrix(tau,xTest)
outputTest = model(A_test,THETA)



In [15]:
# R2 for raw train data
R2_train = r2_score(tTrain.reshape(-1, 1),outputTrain.reshape(-1, 1))
print(R2_train)

0.7818092530573167


In [16]:
# MSE for raw train data
MSE_train = mean_squared_error(tTrain.reshape(-1, 1),outputTrain.reshape(-1, 1))
print(MSE_train)


56903.271268213015


In [17]:
# R2 for raw test data
R2_test = r2_score(tTest.reshape(-1, 1),outputTest.reshape(-1, 1))
print(R2_test)


0.7965078657888922


In [18]:

# MSE for raw test data
MSE_test = mean_squared_error(tTest.reshape(-1, 1),outputTest.reshape(-1, 1))
print(MSE_test)


57518.43864908027


In [19]:
THETA

array([[ 4.51642758e-02,  2.21471542e-01],
       [ 8.38437768e-01,  1.23893727e+00],
       [ 3.19812643e-03, -1.22669404e-02],
       [ 3.80654015e-01,  1.02722627e+00],
       [ 2.89710047e-03,  4.54033384e-03],
       [-2.98960141e-04, -5.96018931e-04]])